In [100]:
import shapely as shp
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [101]:
accidents = pd.read_pickle('../../data/interim/features/accidents.pickle')
accidents['geometry'] = list(zip(accidents.lon, accidents.lat))
accidents['geometry'] = accidents.geometry.apply(shp.geometry.Point)
accidents = gpd.GeoDataFrame(accidents, geometry='geometry')
accidents.crs = {'init' :'epsg:4326'}
accidents = accidents.to_crs('+proj=lcc +lat_1=29.3 +lat_2=30.7 +lat_0=28.5 +lon_0=-91.33333333333333 +x_0=1000000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs ')

In [102]:
roads = pd.read_pickle('../../data/interim/features/roads.pickle')

In [103]:
neighborhoods = pd.read_pickle('../../data/interim/nhoods.pickle')

In [104]:
accidents_in_neighborhoods = gpd.sjoin(accidents, neighborhoods)

In [105]:
accidents_in_neighborhoods = accidents_in_neighborhoods[['TimeCreate', 'lat', 'lon', 'street_1', 'street_2', 'geometry', 'index_right', 'GNOCDC_LAB']]

In [106]:
accidents_in_neighborhoods.columns = ['TimeCreate', 'lat', 'lon', 'street_1', 'street_2', 'geometry', 'nhood_id', 'nhood']

In [108]:
accidents_in_neighborhoods['ItemNumber'] = accidents_in_neighborhoods.index

In [110]:
accidents_joined_1 = pd.merge(accidents_in_neighborhoods,
                              roads,
                              left_on=['street_1', 'nhood_id'],
                              right_on=['fullnameab', 'nhood_id'])

In [154]:
accidents_joined_1[['ItemNumber', 'street_1', 'nhood_id', 'nhood_x', 'nhood_y', 'segment_id']].to_csv('../../../../../Temp/accidents')

In [156]:
roads.to_csv('../../../../../Temp/roads.csv')

In [114]:
accidents_in_neighborhoods[accidents_in_neighborhoods.ItemNumber.isin(accidents_joined_1.ItemNumber)]

,TimeCreate,lat,lon,street_1,street_2,geometry,nhood_id,nhood,ItemNumber
A0015112,2012-01-01 00:51:44,29.951019,-90.066144,Canal St,Tchoupitoulas St,POINT (3682128.000033329 529898.9999998047),804,CENTRAL BUSINESS DISTRICT,A0015112
A0060112,2012-01-01 06:12:22,29.949768,-90.071857,Carondelet St,Poydras St,POINT (3680324.000033329 529423.9999998009),804,CENTRAL BUSINESS DISTRICT,A0060112
A0076612,2012-01-01 10:26:21,29.946760,-90.075343,Julia St,O'Keefe Ave,POINT (3679232.00003333 528317.9999998074),804,CENTRAL BUSINESS DISTRICT,A0076612
A0088012,2012-01-01 12:27:08,29.954273,-90.070997,Baronne St,Canal St,POINT (3680578.000033327 531064.9999998105),804,CENTRAL BUSINESS DISTRICT,A0088012
A0104112,2012-01-01 15:24:45,29.950648,-90.075606,Loyola Ave,Poydras St,POINT (3679133.000033331 529730.9999998068),804,CENTRAL BUSINESS DISTRICT,A0104112
A0242612,2012-01-02 16:35:53,29.951987,-90.075845,Loyola Ave,Perdido St,POINT (3679052.000033328 530216.9999998058),804,CENTRAL BUSINESS DISTRICT,A0242612
A0346012,2012-01-03 11:31:04,29.959375,-90.076043,Iberville St,N Robertson St,POINT (3678960.000033327 532902.9999998077),804,CENTRAL BUSINESS DISTRICT,A0346012
A0349912,2012-01-03 12:09:04,29.954273,-90.070997,Baronne St,Canal St,POINT (3680578.000033327 531064.9999998105),804,CENTRAL BUSINESS DISTRICT,A0349912
A0352712,2012-01-03 12:34:14,29.953027,-90.077932,La Salle St,Perdido St,POINT (3678387.000033328 530587.9999998157),804,CENTRAL BUSINESS DISTRICT,A0352712
A0359912,2012-01-03 13:29:19,29.949987,-90.069151,Camp St,Natchez St,POINT (3681180.00003333 529512.9999998079),804,CENTRAL BUSINESS DISTRICT,A0359912
